In [1]:
# from IPython.display import clear_output
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np
# import os
# import torch
import random
# import seaborn as sns
# from sklearn.metrics import accuracy_score, f1_score
# from PIL import Image
# from functools import partial
# import torchvision
# from torch import nn
# from tqdm import tqdm
# from collections import Counter
# from torch.utils.data.sampler import WeightedRandomSampler
# from torch.utils.data import Dataset, DataLoader
# from torchvision import transforms, datasets
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
import time


def ml_for_eye_care_demo(dzn_photo):    
    time.sleep(5)
    return random.randrange(0, 2, 1)

# def ml_for_eye_care(dzn_photo):
#     def seed_everything(seed):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True

#     SEED = 999
#     seed_everything(SEED)

#     DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     print(f"Using device: {DEVICE}")

#     # Data

#     # A clinician has rated each image for the severity of diabetic retinopathy on a scale of 0 to 4:
#     # - **0 - No DR**
#     # - **1 - Mild**
#     # - **2 - Moderate**
#     # - **3 - Severe**
#     # - **4 - Proliferative DR**

#     # Images may contain artifacts, be out of focus, underexposed, or overexposed. 

#     # - train.csv - the training labels
#     # - test.csv - the test set (you must predict the diagnosis value for these variables)
#     # - sample_submission.csv - a sample submission file in the correct format
#     # - train.zip - the training set images
#     # - test.zip - the public test set images

#     # Dowload directly from kaggle (kaggle.json is needed for API - put it in `/content/`):

#     ! pip install kaggle
#     ! mkdir ~/.kaggle
#     ! cp kaggle.json ~/.kaggle/
#     ! chmod 600 ~/.kaggle/kaggle.json

#     clear_output()

#     ! kaggle competitions download -c aptos2019-blindness-detection

#     ! unzip aptos2019-blindness-detection
#     ! rm aptos2019-blindness-detection.zip

#     clear_output()

#     # EDA

#     train_dir = os.path.join('.','train_images/')
#     df = pd.read_csv(os.path.join('.', 'train.csv'))

#     df['path'] = df['id_code'].map(lambda x: os.path.join(train_dir,'{}.png'.format(x)))
#     df = df.drop(columns=['id_code'])
#     df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe

#     df.head(10)

#     len_df = len(df)
#     print(f"There are {len_df} images in df.")

#     df.diagnosis.value_counts() 

#     df['diagnosis'].hist(figsize = (10, 5))

#     im = Image.open(df['path'][1])
#     width, height = im.size
#     print(width,height) 
#     im.show()

#     plt.imshow(np.asarray(im))

#     # long run
#     for i in range(len_df):
#         img = np.array(Image.open(df['path'][i])).shape

#         if img[2] != 3:
#           print(img)

#     # Load data

#     RESCALE_SIZE = 384, 384
#     BATCH_SIZE = 32

#     plt.rcParams["figure.figsize"] = 15, 10

#     train_df, val_df = train_test_split(df, test_size=0.70,
#                                         stratify=df.diagnosis)

#     # ДЛЯ СКОРОСТИ
#     #train_df, val_df = train_test_split(train_df, test_size=0.25,
#     #                                    stratify=train_df.diagnosis)

#     train_df.reset_index(inplace=True)
#     val_df.reset_index(inplace=True)

#     class BlindDataset(Dataset):
#         def __init__(self, df, mode="train") -> None:
#             super().__init__()

#             self.df = df
#             self.len_ = len(self.df.path)
#             self.mode = mode

#         def __len__(self):
#             return self.len_

#         def load_sample(self, file):
#             image = Image.open(file)
#             image.load()
#             return image

#         def __getitem__(self, index):
#             if self.mode == "train":
#                 transform = transforms.Compose([
#                     transforms.Resize(RESCALE_SIZE),
#                     transforms.RandomAffine(
#                         degrees=(-180, 180),
#                         scale=(0.8, 1.2),
#                         shear=(0.8, 1.2),
#                         ),
#                     transforms.RandomHorizontalFlip(p=0.5),
#                     transforms.RandomVerticalFlip(p=0.5),
#                     transforms.ColorJitter(brightness=(0.2), 
#                         contrast=(0.2),
#                         hue=(0.1),
#                         saturation=(0.2),
#                         ),
#                     transforms.ToTensor(),
#                     transforms.Normalize([0.485, 0.456, 0.406], 
#                                          [0.229, 0.224, 0.225]),
#                 ])

#             else:
#                 transform = transforms.Compose([
#                     transforms.Resize(RESCALE_SIZE),
#                     transforms.CenterCrop(384),
#                     transforms.ToTensor(),
#                     transforms.Normalize([0.485, 0.456, 0.406], 
#                                          [0.229, 0.224, 0.225]),
#                 ])

#             tensor_image = self.load_sample(self.df.path[index])
#             tensor_image = transform(tensor_image)

#             return tensor_image, self.df.diagnosis[index]


#     class_numbers = Counter(train_df.diagnosis)
#     samples_weight = torch.tensor([1 / class_numbers[i] for i in train_df.diagnosis])

#     trainset = BlindDataset(train_df)
#     valset = BlindDataset(val_df, mode="val")

#     sampler = WeightedRandomSampler(samples_weight.type("torch.DoubleTensor"), len(samples_weight))

#     trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
#                              sampler=sampler,
#                              #shuffle=True, 
#                              )
#     valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)


#     psum    = torch.tensor([0.0, 0.0, 0.0])
#     psum_sq = torch.tensor([0.0, 0.0, 0.0])

#     # loop through images
#     for inputs, y in tqdm(trainloader):
#         psum    += inputs.sum(axis        = [0, 2, 3])
#         psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])

#     # pixel count
#     count = len(df) * RESCALE_SIZE[0] * RESCALE_SIZE[0]

#     # mean and std
#     total_mean = psum / count
#     total_var  = (psum_sq / count) - (total_mean ** 2)
#     total_std  = torch.sqrt(total_var)

#     # output
#     print('mean: '  + str(total_mean))
#     print('std:  '  + str(total_std))

#     # 224x224:
#     #mean: tensor([0.3056, 0.1627, 0.0532])
#     #std:  tensor([0.2992, 0.1616, 0.0764])

#     # 728x728: 
#     #mean: tensor([0.3056, 0.1627, 0.0532])
#     #std:  tensor([0.2997, 0.1620, 0.0768])

#     # 450x450:
#     # mean: tensor([0.3057, 0.1628, 0.0533])
#     # std:  tensor([0.2996, 0.1619, 0.0766])

#     ### Класс для тренировки и валидации

#     class Trainer:
#         def __init__(self, model, epochs, criterion,
#                      optimizer, trainloader,
#                      validloader, device, metric,
#                      validate=True, scheduler=None,
#                      verbose=2):

#             self.model = model.to(device)
#             self.epochs = epochs
#             self.criterion = criterion
#             self.optimizer = optimizer
#             self.trainloader = trainloader
#             self.validloader = validloader
#             self.device = device
#             self.metric = metric
#             self.validate = validate
#             self.verbose = verbose
#             self.scheduler = scheduler
#             self.get_probs = nn.Softmax(dim=0)
#             self.train_losses = []
#             self.valid_losses = []
#             self.train_metrics = []
#             self.valid_metrics = []

#         def fit(self, epochs=None):
#             if epochs is None:
#                 epochs = self.epochs

#             for epoch in range(epochs):
#                 train_loss, train_metric = self._train(self.trainloader)
#                 self.train_losses.append(train_loss)
#                 self.train_metrics.append(train_metric)

#                 if self.validate:
#                     val_loss, val_metric = self._validate(self.validloader)
#                     self.valid_losses.append(val_loss)
#                     self.valid_metrics.append(val_metric)

#                     if self.scheduler is not None:
#                         self.scheduler.step(val_loss)
#                 else:
#                     val_loss = "NO"

#                 if self.verbose > 0:
#                     print()
#                     print(f"Epoch {epoch+1} out of {epochs}: Train loss = {train_loss}, validation loss = {val_loss} \n\
#                                              Train metric = {train_metric}, validation metric = {val_metric}")
#                     print()

#             return self

#         def _train(self, loader):
#             self.model.train()
#             epoch_loss = 0
#             epoch_preds = []
#             epoch_targets = []
#             for i, (inputs, targets) in enumerate(loader):
#                 inputs, targets = inputs.to(self.device), targets.to(self.device)
#                 out = self.model(inputs)
#                 loss = self.criterion(out, targets)
#                 epoch_loss += loss.item()
#                 self.optimizer.zero_grad()
#                 loss.backward()

#                 if self.verbose > 1:
#                     print(f"\rTraining: batch {i+1} out of {len(loader)}", end="")

#                 self.optimizer.step()

#                 out = self.get_probs(out)
#                 _, preds = torch.max(out.data, 1)
#                 epoch_preds += list(preds.cpu())
#                 epoch_targets += list(targets.detach().cpu())

#                 self._clear_vram(inputs, targets, out)

#             epoch_loss = epoch_loss/len(loader)
#             epoch_metric = self.metric(epoch_targets, epoch_preds)
#             print("\n", end="")

#             return epoch_loss, epoch_metric

#         def _validate(self, loader):
#             self.model.eval()
#             epoch_loss = 0
#             epoch_preds = []
#             epoch_targets = []
#             with torch.no_grad():
#                 for i, (inputs, targets) in enumerate(loader):
#                     inputs, targets = inputs.to(self.device), targets.to(self.device)              
#                     out = self.model(inputs)
#                     loss = self.criterion(out, targets)

#                     if self.verbose > 1:
#                         print(f"\rValidation: batch {i+1} out of {len(loader)}", end="")

#                     epoch_loss += loss.item()
#                     out = self.get_probs(out)
#                     _, preds = torch.max(out.data, 1)
#                     epoch_preds += list(preds.cpu())
#                     epoch_targets += list(targets.detach().cpu())

#                     self._clear_vram(inputs, targets, out)

#             epoch_loss = epoch_loss/len(loader)
#             epoch_metric = self.metric(epoch_targets, epoch_preds)
#             print("\n", end="")

#             return epoch_loss, epoch_metric

#         def _clear_vram(self, inputs, labels, outputs):
#             inputs = inputs.to("cpu")
#             labels = labels.to("cpu")
#             outputs = outputs.to("cpu")
#             del inputs, labels, outputs
#             torch.cuda.empty_cache()

#     def predict(model, loader):
#         model.eval()
#         predictions = []
#         targ = []

#         with torch.no_grad():
#             for i, (inputs, targets) in enumerate(loader):
#                 inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)              
#                 out = model(inputs)
#                 out = nn.functional.softmax(out, dim=-1)
#                 _, preds = torch.max(out.data, 1)
#                 predictions += list(preds)
#                 targ += list(targets)

#         return predictions, targ


#     def calculate_metric(model, loader, metric=None):
#         if metric is None:
#             metric = accuracy_score

#         preds, targets = predict(model, loader)
#         preds = [i.item() for i in preds]
#         targets = [i.item() for i in targets]

#         return metric(targets, preds)

#     # vgg19


#     model = torchvision.models.vgg19_bn(pretrained=True)

#     N_CLASSES = 5
#     num_features = 4096
#     model.classifier[6] = nn.Linear(num_features, N_CLASSES)

#     model = model.to(DEVICE)

#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=3, factor=0.5)

#     trainer = Trainer(model, 10, criterion, optimizer, trainloader, valloader, 
#                       DEVICE, accuracy_score,
#                       scheduler=scheduler)
#     trainer.fit()

#     sns.lineplot(y=trainer.train_losses, x=range(1, 11), label="Train")
#     sns.lineplot(y=trainer.valid_losses, x=range(1, 11), label="Validation")

#     plt.show()


#     f1_macro = partial(f1_score, average="macro")
#     f1_micro = partial(f1_score, average="micro")

#     calculate_metric(model, valloader)
#     #224x224 batch 64 - .8024017467248908
#     # тоже, только без шаффла 0.7150655021834061

#     calculate_metric(model, valloader, f1_macro)
#     #224x224 batch 64 - .5190434141012477
#     # тоже, только без шаффла 0.5748944091876842

#     calculate_metric(model, valloader, f1_micro)
#     #224x224 batch 64 - 0.8024017467248908
#     # тоже, только без шаффла 

#     # save model
#     torch.save(model.state_dict(), '/content/drive/MyDrive/models/vgg19.txt')

#     # DenseNet169

#     model = torchvision.models.densenet169(pretrained=True)

#     N_CLASSES = 5
#     num_features = 1664
#     model.classifier = nn.Linear(num_features, N_CLASSES)
#     model = model.to(DEVICE)

#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=3, factor=0.5)

#     trainer = Trainer(model, 10, criterion, optimizer, trainloader, valloader, 
#                       DEVICE, accuracy_score,
#                       scheduler=scheduler)
#     trainer.fit()

#     sns.lineplot(y=trainer.train_losses, x=range(1, 11), label="Train")
#     sns.lineplot(y=trainer.valid_losses, x=range(1, 11), label="Validation")

#     plt.show()

#     calculate_metric(model, valloader)
#     #224x224 batch 64 - 0.8144104803493449
#     # тоже, только без шаффла 0.7641921397379913

#     calculate_metric(model, valloader, f1_macro)
#     #224x224 batch 64 - 0.6560790065258206
#     # тоже, только без шаффла 0.6223088864604981

#     calculate_metric(model, valloader, f1_micro)
#     #224x224 batch 64 - 0.8144104803493449
#     # тоже, только без шаффла 0.7641921397379913

#     # save model
#     torch.save(model.state_dict(), '/content/drive/MyDrive/models/densnet169.txt')

#     # DenseNet201

#     model = torchvision.models.densenet201(pretrained=True)

#     N_CLASSES = 5
#     num_features = 1920
#     model.classifier = nn.Linear(num_features, N_CLASSES)
#     model = model.to(DEVICE)

#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=3, factor=0.5)

#     trainer = Trainer(model, 10, criterion, optimizer, trainloader, valloader, 
#                       DEVICE, accuracy_score,
#                       scheduler=scheduler)
#     trainer.fit()

#     sns.lineplot(y=trainer.train_losses, x=range(1, 11), label="Train")
#     sns.lineplot(y=trainer.valid_losses, x=range(1, 11), label="Validation")

#     plt.show()

#     calculate_metric(model, valloader)

#     calculate_metric(model, valloader, f1_macro)

#     calculate_metric(model, valloader, f1_micro)

#     torch.save(model.state_dict(), '/content/drive/MyDrive/models/densnet201.txt')

#     # RESNET34

#     model = torchvision.models.resnet34(pretrained=True)

#     N_CLASSES = 5
#     num_features = 512
#     model.fc = nn.Linear(num_features, N_CLASSES)
#     model = model.to(DEVICE)

#     criterion = nn.CrossEntropyLoss()
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=3, factor=0.5)

#     trainer = Trainer(model, 10, criterion, optimizer, trainloader, valloader, 
#                       DEVICE, accuracy_score,
#                       scheduler=scheduler)
#     trainer.fit()

#     sns.lineplot(y=trainer.train_losses, x=range(1, 11), label="Train")
#     sns.lineplot(y=trainer.valid_losses, x=range(1, 11), label="Validation")

#     plt.show()

#     calculate_metric(model, valloader)
#     #224x224 batch 64 - 0.8144104803493449
#     # тоже, только без шаффла 0.7641921397379913

#     calculate_metric(model, valloader, f1_macro)
#     #224x224 batch 64 - 0.6560790065258206
#     # тоже, только без шаффла 0.6223088864604981

#     calculate_metric(model, valloader, f1_micro)
#     #224x224 batch 64 - 0.8144104803493449
#     # тоже, только без шаффла 0.7641921397379913

#     # save model
#     torch.save(model.state_dict(), '/content/drive/MyDrive/models/resnet34.txt')

#     # Предсказание

#     test_dir = os.path.join('.','test_images/')
#     df_t = pd.read_csv(os.path.join('.', 'test.csv'))

#     df_t['path'] = df_t['id_code'].map(lambda x: os.path.join(test_dir,'{}.png'.format(x)))
#     df_t = df_t.drop(columns=['id_code'])
#     df_t = df_t.sample(frac=1).reset_index(drop=True) #shuffle dataframe

#     df_t.head(10)

#     #predictions, targ

#     #df = pd.DataFrame(submission, columns=["id_code", "diagnosis"])
#     #df.to_csv("submission.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '.\\train.csv'